In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
!pip install transformers
import pandas as pd
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import transformers
from transformers import pipeline
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
train_path='/content/drive/MyDrive/News_Articles_Summarization/train.csv'
test_path='/content/drive/MyDrive/News_Articles_Summarization/test.csv'
train_df = pd.read_csv(train_path, nrows = 5)
test_df = pd.read_csv(test_path, nrows = 5)
train_df = train_df.drop(['id'], axis=1)
test_df = test_df.drop(['id'], axis=1)
train_df

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [29]:
train_df['article'][1]

'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a payment for setting up the meetings," federal prosecutors said in a statement. The complai

In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     5 non-null      object
 1   highlights  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [51]:
sent = []
sent_count = 0
for sentence in train_df['article']:
    sent_count += 1
    sentence_tokens = word_tokenize(sentence)
    sent.append(sentence_tokens)
    # if sent_count >= 1:
    #     break
sent

[['By',
  '.',
  'Associated',
  'Press',
  '.',
  'PUBLISHED',
  ':',
  '.',
  '14:11',
  'EST',
  ',',
  '25',
  'October',
  '2013',
  '.',
  '|',
  '.',
  'UPDATED',
  ':',
  '.',
  '15:36',
  'EST',
  ',',
  '25',
  'October',
  '2013',
  '.',
  'The',
  'bishop',
  'of',
  'the',
  'Fargo',
  'Catholic',
  'Diocese',
  'in',
  'North',
  'Dakota',
  'has',
  'exposed',
  'potentially',
  'hundreds',
  'of',
  'church',
  'members',
  'in',
  'Fargo',
  ',',
  'Grand',
  'Forks',
  'and',
  'Jamestown',
  'to',
  'the',
  'hepatitis',
  'A',
  'virus',
  'in',
  'late',
  'September',
  'and',
  'early',
  'October',
  '.',
  'The',
  'state',
  'Health',
  'Department',
  'has',
  'issued',
  'an',
  'advisory',
  'of',
  'exposure',
  'for',
  'anyone',
  'who',
  'attended',
  'five',
  'churches',
  'and',
  'took',
  'communion',
  '.',
  'Bishop',
  'John',
  'Folda',
  '(',
  'pictured',
  ')',
  'of',
  'the',
  'Fargo',
  'Catholic',
  'Diocese',
  'in',
  'North',
  'Dak

In [52]:
import re
otherwords = ['by','associated','press','published','est','october','updated'] # add other words here if you think it should not be in the sentence

In [55]:
stopwords_english = stopwords.words('english') 
punctuation = string.punctuation
punctuation = punctuation.replace('.','')


formatted_sent = []
for sentence in sent:
    formatted_words = []
    for word in sentence:
        if word not in stopwords_english and word not in punctuation:  #Removes word with less than 2 characters, present in english stop words or is a punctuation
            formatted_words.append(word.lower())
    formatted_sent.append(formatted_words)
print(formatted_sent)

[['by', '.', 'associated', 'press', '.', 'published', '.', '14:11', 'est', '25', 'october', '2013', '.', '.', 'updated', '.', '15:36', 'est', '25', 'october', '2013', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', 'october', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'churches', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'says', 'risk', 'low', 'officials', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john'

In [57]:
formatted_sent_two = []
for sentence in formatted_sent:
    formatted_words = []
    for word in sentence:
        if word not in otherwords and not re.search('\d{2}:\d{2}',word) and not re.search('\d{2,4}',word):  #Removes time, year and few other irrelevant terms.
            formatted_words.append(word)
    formatted_sent_two.append(formatted_words)
print(formatted_sent_two)

[['.', '.', '.', '.', '.', '.', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'churches', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'says', 'risk', 'low', 'officials', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john', 'folda', 'taking', 'time', 'diagnosed', 'hepatitis', 'a', '.', 'the', 'diocese', 'says', 'contracted', 'infection', 'contaminated', 'food', 'att

In [58]:
lemma_sent = []

lemma = WordNetLemmatizer()

for sentence in formatted_sent_two:
    lemma_words = []
    for word in sentence:
        lemma_word = lemma.lemmatize(word)
        lemma_words.append(lemma_word)
    lemma_sent.append(lemma_words)
print(lemma_sent)

[['.', '.', '.', '.', '.', '.', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundred', 'church', 'member', 'fargo', 'grand', 'fork', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'church', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundred', 'church', 'member', 'fargo', 'grand', 'fork', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'say', 'risk', 'low', 'official', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john', 'folda', 'taking', 'time', 'diagnosed', 'hepatitis', 'a', '.', 'the', 'diocese', 'say', 'contracted', 'infection', 'contaminated', 'food', 'attending', 'c

In [59]:
final_sentence = []
formatted_sent_list = []


for sentence in lemma_sent:
    words = ' '.join([str(word) for word in sentence])
    final_sentence.append(words)
formatted_sent_list.append(' '.join(final_sentence))
print(final_sentence)

[". . . . . . . the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early . the state health department issued advisory exposure anyone attended five church took communion . bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a . state immunization program manager molly howell say risk low official feel 's important alert people possible exposure . the diocese announced monday bishop john folda taking time diagnosed hepatitis a . the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month . symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort . fargo catholic diocese north dakota pictured bishop located .", "cnn -- ralph mata internal affair lieutenant miami-dade police department working division investigates alleg

In [25]:
# sentence = str(final_sentence[0][14:])
# print(sentence)

go catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early the state health department issued advisory exposure anyone attended five church took communion bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a state immunization program manager molly howell say risk low official feel 's important alert people possible exposure the diocese announced monday bishop john folda taking time diagnosed hepatitis a the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort fargo catholic diocese north dakota pictured bishop located


In [39]:
# del sent
# del formatted_sent
# del formatted_sent_two
# del lemma_sent

In [40]:
# word_count = {}
# for word in word_tokenize(sentence):
#     if word not in ['.']:
#         if word in word_count.keys():
#             word_count[word] +=1
#         else:
#             word_count[word] = 1
# print(word_count)

In [41]:
# fin_max = max(word_count, key=word_count.get)
# print("Maximum value word:",fin_max)

In [42]:
# max_val = word_count[fin_max]
# print("Maximum value:",max_val)

In [43]:
# for key,value in word_count.items():
#     word_count[key] = word_count[key]/max_val
# print(word_count)

In [45]:
# for sentences in sent_tokenize(sentence):
#     print(sentences)

In [46]:
# sentence_score = {}

# for sentences in sent_tokenize(sentence):
#     sentence_score[sentences] = 0
#     for word in sentences:
#         if word in word_count.keys():
#             sentence_score[sentences]+=word_count[word]
# print(sentence_score)

In [61]:
# Summarization
summarizer = pipeline("summarization", model="bert-base-uncased", tokenizer="bert-base-uncased")

summary_list = []
for article in formatted_sent_list:
    sentence_list = sent_tokenize(article)
    summary = summarizer(sentence_list, min_length=0, max_length=10)
    summary_text = ''
    for sentence in summary:
        summary_text += sentence['summary_text'] + ' '
    summary_list.append(summary_text)

# Concatenate summaries
full_summary = '$'
for summary in summary_list:
    full_summary += summary + 'KHATAM'
print(full_summary)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The model 'BertForMaskedLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration'

$........ ........ ........ ........ ........ ........ ........ the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early. - the state health department issued advisory exposure anyone attended five church took communion. the bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a. minnesota state immunization program manager molly howell say risk low official feel ' s important alert people possible exposure. safety the diocese announced monday bishop john folda taking time diagnosed hepatitis a. the the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month. he symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort. vomiting fargo catholic diocese north dakota pictured bishop located. see cnn - - ralph mata internal 

In [62]:
formatted_sent_list

[". . . . . . . the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early . the state health department issued advisory exposure anyone attended five church took communion . bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a . state immunization program manager molly howell say risk low official feel 's important alert people possible exposure . the diocese announced monday bishop john folda taking time diagnosed hepatitis a . the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month . symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort . fargo catholic diocese north dakota pictured bishop located . cnn -- ralph mata internal affair lieutenant miami-dade police department working division investigates allegati

In [143]:
actual_summaries = test_df['highlights'].tolist()
actual_summaries[4]

"Tell-all interview with the reality TV star, 69, will air on Friday April 24 .\nIt comes amid continuing speculation about his transition to a woman and following his involvement in a deadly car crash in February .\nThe interview will also be one of Diane Sawyer's first appearances on television following the sudden death of her husband last year ."

In [70]:
x = [{'id':"zzz"}, "YYY"]
x[0]

{'id': 'zzz'}

In [86]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Initialize the predicted and actual summaries
summary_list = ["The bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early. - the state health department issued advisory exposure anyone attended five church took communion. the bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a. minnesota state immunization program manager molly howell say risk low official feel ' s important alert people possible exposure. safety the diocese announced monday bishop john folda taking time diagnosed hepatitis a. the the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month. he symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort. vomiting fargo catholic diocese north dakota pictured bishop located."]
# actual_summaries = ["Experts question if  packed out planes are putting passengers at risk . U.S consumer advisory group says minimum space must be stipulated . Safety tests conducted on planes with more leg room than airlines offer ."]
actual_summaries = ["The bishop John Folda, of North Dakota, is taking time off after being diagnosed . He contracted the infection through contaminated food in Italy . Church members in Fargo, Grand Forks and Jamestown could have been exposed ."]
# summary_list = actual_summaries
# Get the actual summaries from the test dataset
# actual_summaries = test_df['highlights'][0]

# Print the predicted and actual summaries for the first article in the test dataset
print('Predicted summary:', summary_list[0])
print('Actual summary:', actual_summaries)

# Calculate and print the accuracy score
accuracy = accuracy_score(actual_summaries, summary_list)
print('Accuracy:', accuracy)

# # Calculate and print the confusion matrix
# conf_matrix = confusion_matrix(actual_summaries, summary_list)
# print('Confusion matrix:')
# # print(conf_matrix)

Predicted summary: The bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early. - the state health department issued advisory exposure anyone attended five church took communion. the bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a. minnesota state immunization program manager molly howell say risk low official feel ' s important alert people possible exposure. safety the diocese announced monday bishop john folda taking time diagnosed hepatitis a. the the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month. he symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort. vomiting fargo catholic diocese north dakota pictured bishop located.
Actual summary: ['The bishop John Folda, of North Dakota, is taking time off 